In [1]:
val datadir = "/opt/lsde/dataset-sf100"
val a1 = 2811
val a2 = 1990
val a3 = 5183
val a4 = 1749
val lo = 409
val hi = 509

Intitializing Scala interpreter ...

Spark Web UI available at http://centos8.linuxvmimages.local:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1600293370231)
SparkSession available as 'spark'


datadir: String = /opt/lsde/dataset-sf100
a1: Int = 2811
a2: Int = 1990
a3: Int = 5183
a4: Int = 1749
lo: Int = 409
hi: Int = 509


In [2]:
 // load the three tables
  val person   = spark.read.format("csv").option("header", "true").option("delimiter", "|").option("inferschema", "true").
                       load(datadir + "/person.*csv.*").cache()
  val interest = spark.read.format("csv").option("header", "true").option("delimiter", "|").option("inferschema", "true").
                       load(datadir + "/interest.*csv.*").cache()
  val knows    = spark.read.format("csv").option("header", "true").option("delimiter", "|").option("inferschema", "true").
                       load(datadir + "/knows.*csv.*").cache()

person: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [personId: bigint, firstName: string ... 7 more fields]
interest: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [personId: bigint, interest: int]
knows: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [personId: bigint, friendId: bigint]


In [12]:
knows.show()

+--------------+-------------+
|      personId|     friendId|
+--------------+-------------+
|14293665346767|1099528496577|
|14293665346767|2199033865163|
|14293665346767|2199035357154|
|14293665346767|2199038966227|
|14293665346767|2199039507672|
|14293665346767|2199041731939|
|14293665346767|3298550330337|
|14293665346767|4398057217998|
|14293665346767|4398057338719|
|14293665346767|4398063911513|
|14293665346767|5497568743267|
|14293665346767|5497570002273|
|14293665346767|5497572367152|
|14293665346767|5497574384945|
|14293665346767|6597081151943|
|14293665346767|6597086760755|
|14293665346767|6597087568354|
|14293665346767|6597087655106|
|14293665346767|6597088991061|
|14293665346767|7696593878361|
+--------------+-------------+
only showing top 20 rows



In [13]:
interest.show()

+--------+--------+
|personId|interest|
+--------+--------+
|     420|       1|
|    7489|       3|
|    7489|     138|
|    7489|     142|
|    7489|     973|
|    7489|    1533|
|    7489|    1669|
|    7489|    2017|
|    7489|    2026|
|    7489|    2040|
|    7489|    2089|
|    7489|    2114|
|    7489|    2800|
|    7489|    2825|
|    7489|    2830|
|    7489|    2852|
|    7489|    2898|
|    7489|    2939|
|    7489|    2987|
|    7489|    2995|
+--------+--------+
only showing top 20 rows



In [ ]:
person.show()

In [3]:
  // select the relevant (personId, interest) tuples, and add a boolean column "nofan" (true iff this is not a a1 tuple)
  val focus    = interest.filter($"interest" isin (a1, a2, a3, a4)).
                          withColumn("nofan", $"interest".notEqual(a1))

focus: org.apache.spark.sql.DataFrame = [personId: bigint, interest: int ... 1 more field]


In [7]:
focus.show()

+--------+--------+-----+
|personId|interest|nofan|
+--------+--------+-----+
|   55234|    1749| true|
|   62883|    1990| true|
|   85063|    1749| true|
|   86156|    5183| true|
|  107471|    1749| true|
|  142797|    1990| true|
|  142797|    2811|false|
|  153142|    2811|false|
|  178578|    1990| true|
|  187435|    1990| true|
|  187435|    2811|false|
|  208544|    1990| true|
|  208544|    2811|false|
|  229182|    2811|false|
|  246872|    1749| true|
|  264483|    1990| true|
|  264483|    2811|false|
|  278363|    2811|false|
|  278363|    5183| true|
|  280603|    1990| true|
+--------+--------+-----+
only showing top 20 rows



In [4]:
  // compute person score (#relevant interests): join with focus, groupby & aggregate. Note: nofan=true iff person does not like a1
  val scores   = person.join(focus, "personId").
                        groupBy("personId", "locatedIn", "birthday").
                        agg(count("personId") as "score", min("nofan") as "nofan")

scores: org.apache.spark.sql.DataFrame = [personId: bigint, locatedIn: int ... 3 more fields]


In [9]:
scores.show()

+-------------+---------+----------+-----+-----+
|     personId|locatedIn|  birthday|score|nofan|
+-------------+---------+----------+-----+-----+
|      1377470|      220|1985-03-05|    1|false|
|      4673872|      197|1981-05-28|    1|false|
|      6010647|      112|1986-02-07|    1| true|
|      7486409|      126|1988-01-30|    2| true|
|     20161074|      541|1980-08-25|    1| true|
|     22268500|      577|1986-11-24|    2| true|
|     23298065|      627|1988-09-14|    1| true|
|     23529260|      622|1985-03-16|    2| true|
|     33668664|      817|1989-01-19|    1| true|
|     38130496|      951|1980-11-18|    1| true|
|     38574506|      954|1985-04-12|    1| true|
|     39127057|      935|1982-04-14|    1| true|
|     39745737|      959|1987-01-10|    2|false|
|     41637299|     1002|1987-09-19|    1| true|
|     44522417|     1054|1980-02-02|    1|false|
|1099512348497|      284|1980-10-04|    1| true|
|1099519905809|      127|1981-02-20|    1|false|
|1099524258560|     

In [5]:
  // filter (personId, score, locatedIn) tuples with score>1, being nofan, and having the right birthdate
  val cands    = scores.filter($"score" > 0 && $"nofan").
                        withColumn("bday", month($"birthday")*100 + dayofmonth($"birthday")).
                        filter($"bday" >= lo && $"bday" <= hi)

cands: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [personId: bigint, locatedIn: int ... 4 more fields]


In [11]:
cands.printSchema()

root
 |-- personId: long (nullable = true)
 |-- locatedIn: integer (nullable = true)
 |-- birthday: string (nullable = true)
 |-- score: long (nullable = false)
 |-- nofan: boolean (nullable = true)
 |-- bday: integer (nullable = true)



In [12]:
cands.show()

+--------------+---------+----------+-----+-----+----+
|      personId|locatedIn|  birthday|score|nofan|bday|
+--------------+---------+----------+-----+-----+----+
| 1099529980383|      351|1985-04-27|    1| true| 427|
| 2199038780764|      489|1987-04-10|    1| true| 410|
| 2199066237444|     1028|1980-04-21|    1| true| 421|
| 2199068690820|     1074|1984-04-21|    1| true| 421|
| 3298598731794|     1450|1985-04-21|    1| true| 421|
| 4398073613871|      677|1988-05-09|    1| true| 509|
| 4398075302177|      718|1986-04-16|    1| true| 416|
| 5497589137337|      761|1989-05-04|    1| true| 504|
| 5497598693788|      975|1983-04-22|    1| true| 422|
| 5497620003851|     1421|1986-04-12|    1| true| 412|
| 9895653896064|     1158|1983-04-22|    1| true| 422|
|10995120128442|      205|1986-04-20|    2| true| 420|
|12094648220302|      544|1988-04-16|    1| true| 416|
|12094679955379|     1214|1989-04-13|    1| true| 413|
|13194144999863|      305|1986-04-19|    1| true| 419|
|142936575

In [6]:
  // create (personId, ploc, friendId, score) pairs by joining with knows (and renaming locatedIn into ploc)
  val pairs    = cands.select($"personId", $"locatedIn".alias("ploc"), $"score").
                       join(knows, "personId")

pairs: org.apache.spark.sql.DataFrame = [personId: bigint, ploc: int ... 2 more fields]


In [7]:
pairs.show()

+-------------+----+-----+-------------+
|     personId|ploc|score|     friendId|
+-------------+----+-----+-------------+
|1099529980383| 351|    1|     10281936|
|1099529980383| 351|    1|     12033224|
|1099529980383| 351|    1|     12215646|
|1099529980383| 351|    1|     13798960|
|1099529980383| 351|    1|     17444704|
|1099529980383| 351|    1|     30067653|
|1099529980383| 351|    1|     38477637|
|1099529980383| 351|    1|     43744355|
|1099529980383| 351|    1|     47046621|
|1099529980383| 351|    1|     49314529|
|1099529980383| 351|    1|     53289156|
|1099529980383| 351|    1|     56492110|
|1099529980383| 351|    1|     57603292|
|1099529980383| 351|    1|     62010974|
|1099529980383| 351|    1|1099514300386|
|1099529980383| 351|    1|1099521755978|
|1099529980383| 351|    1|1099523049827|
|1099529980383| 351|    1|1099523066182|
|1099529980383| 351|    1|1099523649492|
|1099529980383| 351|    1|1099523852105|
+-------------+----+-----+-------------+
only showing top

In [7]:
  // re-use the scores dataframe to create a (friendId, floc) dataframe of persons who are a fan (not nofan)
  val fanlocs  = scores.filter(!$"nofan").select($"personId".alias("friendId"), $"locatedIn".alias("floc"))

fanlocs: org.apache.spark.sql.DataFrame = [friendId: bigint, floc: int]


In [8]:
  // join the pairs to get a (personId, ploc, friendId, floc, score), and then filter on same location, and remove ploc and floc columns
  val results  = pairs.join(fanlocs, "friendId").
                       filter($"ploc"===$"floc").
                       select($"personId".alias("p"), $"friendId".alias("f"), $"score")

results: org.apache.spark.sql.DataFrame = [p: bigint, f: bigint ... 1 more field]


In [9]:
  // do the bidirectionality check by joining towards knows, and keeping only the (p, f, score) pairs where also f knows p
  val bidir    = results.join(knows.select($"personId".alias("f"), $"friendId"), "f").filter($"p"===$"friendId")

bidir: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [f: bigint, p: bigint ... 2 more fields]


In [10]:
  // keep only the (p, f, score) columns and sort the result
  val ret      = bidir.select($"p", $"f", $"score").orderBy(desc("score"), asc("p"), asc("f"))

ret: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [p: bigint, f: bigint ... 1 more field]


In [11]:
 ret.show(1000) 

+--------------+--------------+-----+
|             p|             f|score|
+--------------+--------------+-----+
|      23749085|      23166928|    2|
|      50823520| 7696632439459|    2|
| 3298594899508|16492734279256|    2|
| 5497600779230|12094670330460|    2|
| 6597123132851|17592239440354|    2|
| 7696635963102|16492729060318|    2|
| 8796106570192| 6597087722069|    2|
| 8796112196963| 2199042458430|    2|
| 8796112196963| 6597088928349|    2|
| 8796145390018|13194192100803|    2|
|10995164908940|14293699756896|    2|
|10995170740188|10995170565820|    2|
|12094671702986|10995160317409|    2|
|12094671702986|13194183642702|    2|
|13194158441785| 2199042139230|    2|
|13194159461967| 7696601299411|    2|
|13194181541341| 8796135011899|    2|
|15393213728561| 6597120875355|    2|
|16492736735821| 4398108821581|    2|
|16492736735821| 9895667050205|    2|
|18691721503709| 4398070033760|    2|
|18691752124106|10995170731206|    2|
|        939359| 2199032216743|    1|
|       8461